# 0. 데이터 업로드 및 전처리

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dasol109","key":"87c73566bec35f1ed3da5ca73b66b5c5"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d tomsezequielrau/global-weekly-spotify-top-200-20172020

  0% 0.00/1.86M [00:00<?, ?B/s]
100% 1.86M/1.86M [00:00<00:00, 31.0MB/s]


In [ ]:
!unzip -q global-weekly-spotify-top-200-20172020.zip

In [ ]:
import pandas as pd
raw_df = pd.read_csv("/content/global_weekly_top_200_2017to2020.csv", index_col=0)
raw_df.info()
df = raw_df.iloc[-200:, :5]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42200 entries, 0 to 42199
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Position    42200 non-null  int64 
 1   Track Name  42191 non-null  object
 2   Artist      42191 non-null  object
 3   Streams     42200 non-null  int64 
 4   URL         42200 non-null  object
 5   filedate    42200 non-null  object
 6   start_week  42200 non-null  object
 7   end_week    42200 non-null  object
 8   track_id    42200 non-null  object
dtypes: int64(2), object(7)
memory usage: 3.2+ MB


# 0. 테스트

In [ ]:
import numpy as np


df = df.reset_index()
df = df.drop(['Position'],axis=1)
df = df.drop(['URL'],axis=1)
df = df.drop(['index'],axis=1)
df.loc[-1] = [0, 0, 0]  # adding a row
df.index = df.index + 1  # shifting index
df.sort_index(inplace=True) 
df.head()

,Track Name,Artist,Streams
0,0,0,0
1,drivers license,Olivia Rodrigo,80764045
2,DÁKITI,Bad Bunny,30189319
3,Blinding Lights,The Weeknd,23094057
4,Mood (feat. iann dior),24kGoldn,22974369


In [ ]:
print(df)

                     Track Name          Artist   Streams
0                             0               0         0
1               drivers license  Olivia Rodrigo  80764045
2                        DÁKITI       Bad Bunny  30189319
3               Blinding Lights      The Weeknd  23094057
4        Mood (feat. iann dior)        24kGoldn  22974369
..                          ...             ...       ...
196                        Hope    XXXTENTACION   4830862
197  rockstar (feat. 21 Savage)     Post Malone   4811482
198  The Less I Know The Better     Tame Impala   4807824
199              Lovesick Girls       BLACKPINK   4796365
200                    Caramelo           Ozuna   4790233

[201 rows x 3 columns]


# 1. 관리자

In [ ]:
class Heap:
  def __init__(self, df_data):
    self.df = df_data.copy()

  def bHeap(self):

    for i in range(len(self.df)//2, 0, -1):
      self.downHeap(self.df.iloc[i])
      
    print(self.df)

  def upHeap(self, p):
    #print("upheap 들어왔어!")
    #print(p)
    #print(self.df.iloc[1])
    
    if p['Streams'] == self.df.iloc[1]['Streams']:
      return
    
    for i, data in self.df.iterrows():
      if data['Streams'] == p['Streams']:
          p_index = i

    if p['Streams'] < self.df.iloc[p_index//2]['Streams']:
        return

    self.df.iloc[p_index] = self.df.iloc[p_index//2]
    self.df.iloc[p_index//2] = p
    self.upHeap(self.df.iloc[p_index//2])


  def downHeap(self, p):
    
    #print("downHeap이에요~")
    #print(self.df)
    for i, data in self.df.iterrows():
      if data['Streams'] == p['Streams']:
          p_index = i

    #print(p)
    #print("p_index는")
    #print(p_index)

    if (len(self.df) <= p_index*2) & (len(self.df) <= (p_index*2+1)):
        #print("첫번째 돌아가는 길이에요~")
        return

    b = self.df.iloc[p_index*2]

    if (len(self.df) > p_index*2+1):
        if (self.df.iloc[p_index*2+1]['Streams'] > b['Streams']):
          b = self.df.iloc[p_index*2+1]

    #print("b는")
    #print(b)

    if p['Streams'] > b['Streams']:
        #print("두번째 돌아가는 길이에요~")
        return

    for i, data in self.df.iterrows():
        if data['Streams'] == b['Streams']:
          b_index = i
      

    self.df.iloc[p_index] = self.df.iloc[b_index]
    self.df.iloc[b_index] = p
    self.downHeap(self.df.iloc[b_index])


  def insert(self, row):
    self.df = self.df.append(row,ignore_index=True)
    p = self.df.iloc[-1]
    self.upHeap(p)


  def search(self, query):
    df_name = self.df[self.df['Track Name'].str.contains(query, case=False, na=False)]
    df_artist = self.df[self.df['Artist'].str.contains(query, case=False, na=False)]

 
    df_result = pd.concat([df_name, df_artist])

    print(df_result)


  def update(self, query):
    df_result = self.search(query)
    try:
      select = int(input("몇 번 노래를 업데이트할까요?: "))
      part = input("어느 부분을 업데이트할까요?: ")
      #만약 Stream이면 upheap하기
      if (part == 'Streams'):
        value = int(input("어느 값으로 업데이트할까요?: "))
        self.df.loc[self.df.index == select, ('Streams')] = value
        p = self.df.iloc[select]
        print(p)
        self.upHeap(p)
      else:
        value = input("어느 값으로 업데이트할까요?: ")
        self.df.loc[self.df.index == select, (part)] = value
      df_sample_4 = self.df.iloc[[select], :]

    except:
      print("잘못된 값을 입력하셨습니다.")
    

    print(df_sample_4)

  def delete(self, query):
    df_result = self.search(query)
    try:
      n = int(input("몇 번 노래를 삭제할까요?: "))

      root = self.df.iloc[n]
      temp = self.df.iloc[-1]

      self.df.drop(index=n, axis=0)
      self.df.iloc[n]= self.df.iloc[-1]
      self.df = self.df.iloc[:-1 , :]
      self.downHeap(self.df.iloc[n])

      print("delete 완료")

    except:
      print("잘못된 값을 입력하셨습니다.")
  

    


  def pr(self):
    music = []
    try:
      n = int(input("몇 개의 노래를 보여드릴까요?: "))

      for data in range(0, n):
        root = self.df.iloc[1]
        temp = self.df.iloc[-1]
        print(root)
        music.append(root)

        self.df.drop(index=1, axis=0)
        self.df.iloc[1]= self.df.iloc[-1]
        self.df = self.df.iloc[:-1 , :]
        self.downHeap(self.df.iloc[1])
      print()
      for i in range(0, n):
        Heap.insert(music[i])
    except:
      print("숫자를 입력해주세요.")


      
Heap = Heap(df)
Heap.bHeap()

                     Track Name          Artist   Streams
0                             0               0         0
1               drivers license  Olivia Rodrigo  80764045
2                        DÁKITI       Bad Bunny  30189319
3               Blinding Lights      The Weeknd  23094057
4        Mood (feat. iann dior)        24kGoldn  22974369
..                          ...             ...       ...
196                        Hope    XXXTENTACION   4830862
197  rockstar (feat. 21 Savage)     Post Malone   4811482
198  The Less I Know The Better     Tame Impala   4807824
199              Lovesick Girls       BLACKPINK   4796365
200                    Caramelo           Ozuna   4790233

[201 rows x 3 columns]


In [ ]:
def admin_main():
  #Heap.insert(pd.DataFrame([['Let it go', 'Elsa', 23094059, '000']], columns=['Track Name', 'Artist', 'Streams', 'URL']))
  #Heap.pr()
  

  while(1):

    print("*"*30)
    print(" c: 추가 \n u: 업데이트 \n s: 검색 \n d: 제거 \n p: 출력 \n q: 종료")
    print("*"*30)
    function = input("무엇을 하고자 하십니까?: ")
    if function == "q":
      break
    elif function == "c":
      try:
        Track_Name = input("노래 제목을 입력해주세요.: ")
        Artist_Name = input("가수 이름을 입력해주세요.: ")
        Streams = int(input("스트리밍 수를 입력해주세요.: "))
        Heap.insert(pd.DataFrame([[Track_Name, Artist_Name, Streams]], columns=['Track Name', 'Artist', 'Streams']))
        print(pd.DataFrame([[Track_Name, Artist_Name, Streams]], columns=['Track Name', 'Artist', 'Streams']))
        print("insert 완료")
      except:
        print("잘못된 값을 입력하셨습니다.")
    elif function == "u":
      try:
        query = input("업데이트할 노래를 입력해주세요.: ")
        Heap.update(query)
      except:
        print("잘못된 값을 입력하셨습니다.")
    elif function == "s":
      try:
        query = input("검색할 노래를 입력해주세요.: ")
        Heap.search(query)
      except:
        print("잘못된 값을 입력하셨습니다.")
    elif function == "p":
      Heap.pr()
    elif function == "d":
      try:
        query = input("삭제할 노래를 입력해주세요.: ")
        Heap.delete(query)
      except:
        print("잘못된 값을 입력하셨습니다.")
    else:
      print("잘못된 입력입니다.")


In [ ]:
admin_main()

******************************
 c: 추가 
 u: 업데이트 
 s: 검색 
 d: 제거 
 p: 출력 
 q: 종료
******************************
무엇을 하고자 하십니까?: p
몇 개의 노래를 보여드릴까요?: e
숫자를 입력해주세요.
******************************
 c: 추가 
 u: 업데이트 
 s: 검색 
 d: 제거 
 p: 출력 
 q: 종료
******************************
무엇을 하고자 하십니까?: p
몇 개의 노래를 보여드릴까요?: 10
Track Name    drivers license
Artist         Olivia Rodrigo
Streams              80764045
Name: 1, dtype: object
Track Name       DÁKITI
Artist        Bad Bunny
Streams        30189319
Name: 1, dtype: object
Track Name    Blinding Lights
Artist             The Weeknd
Streams              23094057
Name: 1, dtype: object
Track Name    Mood (feat. iann dior)
Artist                      24kGoldn
Streams                     22974369
Name: 1, dtype: object
Track Name     BICHOTA
Artist         KAROL G
Streams       21262093
Name: 1, dtype: object
Track Name        positions
Artist        Ariana Grande
Streams            20973589
Name: 1, dtype: object
Track Name    Good Days
Artist       

# 2. 사용자

## 2.1. Ranking

- 최대힙을 이용하여 힙정렬 수행

In [ ]:
Ranking = df.set_index("Streams").iloc[:,1:] # df에서 대강 만듦
Ranking.info()

## 2.2. 사용자 자료구조

- 원형 큐
- 연결리스트

트랙 리스트에서는 이용자가 선택한 음원을 순서대로 원형큐로 관리한다. 원형큐는 연결리스트를 사용하여 연결된 큐(linked queue)로 구현된다. 이때 각 노드는 Track Name, Artist, URL로 구성된다. 큐는 첫번째로 들어오는 데이터를 가장 먼저 내보내는 선입선출(First In First Out)방식을 사용한다. 그 중 원형큐는 큐의 front 포인터와 rear 포인터가 연결되어, Out of Bounds Exception 문제가 발생하지 않고 다시 처음부터 순환되는 구조를 가진다. 큐의 front  포인터는 연결리스트의 맨 앞에 있는 요소를 가리키며, 큐의 rear 포인터는 연결리스트의 맨 뒤에 있는 요소를 가리킨다.
<br><br>

|구분|조건|
|:---:|:---:|
|삽입 위치|rear = (rear+1) mod n|
|공백 상태|front = rear|


<br><br>

In [ ]:
class Queue:   
  def __init__(self):
    self.front = 0
    self.rear = 0
    self.n = 0
    self.array = []
  
  def isEmpty(self):
    return self.front == self.rear
  
  def enqueue(self, item):
    tmp = item
    self.n = self.n + 1
    self.rear = self.rear + 1
    self.array.append(item)
    return tmp
  
  def dequeue(self, index):
    tmp = self.array[index]
    del self.array[index]
    self.rear = self.rear - 1
    self.n = self.n - 1
    return tmp


## 2.3. 사용자 기능
- Create
- Play
- Search
- Delete by Search

Track List에서는 이용자들이 자신의 트랙 리스트를 관리한다. 이용자는 정렬되어 제공되는 음원리스트에서 원하는 음원을 선택하거나, 검색하여 Track List에 넣을 수 있다. 또한 원하지 않은 음원을 Track List에서 삭제할 수 있다. 사용자가 원하는 경우 Play 기능을 통해 음원을 재생할 수 있다. 재생은 일반 재생과 반복 재생 중 선택할 수 있으며, 일반 재생을 선택할 경우 Track List의 처음부터 끝까지의 음원을 출력한다. 반복 재생을 선택할 경우 Track List의 끝까지 음원이 출력된다. 

**1) Create** <br>
이용자가 원하는 음원을 트랙리스트에 새로 삽입하면, *isFull 함수와 enqueue 함수를 통해 큐가 Full인지 확인한 후,* rear에 새로운 음원을 삽입한다.
- isFull 함수를 통해 큐가 Full인지 확인하는 과정 건너뜀

**2) Play** <br>
이용자는 Play 기능을 이용하여 Track List의 음원들을 재생할 수 있다. 이때 일반 재생과 반복 재생을 선택할 수 있다. 일반 재생을 사용한다면 원형큐에서 front에서 rear로 이동하며 음원 정보를 재생한 후, rear에 도착하여 마지막 음원이 출력되고 front로 돌아오는 순간 종료된다. 반면 반복 재생을 사용한다면 원형큐에서 front에서 rear로 이동하며 음원 정보를 재생한 후, rear에 도착하여 마지막 음원이 출력되고 front로 돌아오는 순간 front부터 다시 재생된다.

**3) Search** <br>
이용자는 Search 기능을 사용하여 전체 음원들 중에서 자신이 원하는 음원을 검색할 수 있다. 검색은 Music 데이터 안의 Track Name, Artist를 통해 이루어진다. 검색어와 일치하는 데이터들을 Read를 통해 콘솔에 보여준다.

**4) Delete by Search** <br>
이용자는 원하는 음원을 삭제할 수 있다. Search 기능을 사용하여 트랙리스트 내 음원을 검색한 후, isEmpty, dequeue 함수를 통해 큐가 Empty인지 확인한 후, 음원을 삭제한다.


In [ ]:
def Create(Ranking, TrackList):
  while True:
    idx = input("넣고 싶은 곡의 번호를 입력하세요(1~200, 0: 종료) ")
    if idx.isnumeric()==False:
      print("잘못된 입력입니다.")
    else:
      idx = int(idx)
      if idx>=1 and idx<=200:
        result = TrackList.enqueue(list(Ranking.iloc[idx-1]))
        print(f"{result[0]} {result[1]} 가 추가되었습니다.")
      else:
        break

def Play(TrackList): # 입력한 횟수만큼 트랙 전체 재생
  if TrackList.isEmpty():
    print("트랙 리스트가 비어 있습니다.")
    return
  play = int(input("재생 횟수를 입력하시오. "))
  for i in range(play):
    idx = 0 # TrackList.front
    while True:
      if idx == TrackList.rear:
        break
      print(f"Track Name: {TrackList.array[idx][0]}     | Artist: {TrackList.array[idx][1]}     | URL: {TrackList.array[idx][2]}")
      idx = idx + 1

def Search(TrackList): # 하나의 곡이 여러 번 입력되어 있어도, 가장 처음의 index를 return
  if TrackList.isEmpty():
    print("트랙 리스트가 비어 있습니다.")
    return
  kind = input("어느 것으로 검색하실 건가요?(t: Track Name, a: Artist) ")
  if kind == "t":
    keyword = input("Track Name을 입력하세요. ")
    for i in range(TrackList.rear):
      if keyword == TrackList.array[i][0]:
        tmp = TrackList.array[i]
        print(f"검색 결과: {i}번째  {tmp[0]}     {tmp[1]}")
        return i
  elif kind == "a":
    keyword = input("Artist를 입력하세요. ")
    for i in range(TrackList.rear):
      if keyword == TrackList.array[i][1]:
        tmp = TrackList.array[i]
        print(f"검색 결과: {i}번째  {tmp[0]}     {tmp[1]}")
        return i
  else:
    i = None
    print("잘못된 입력입니다.")
  

def Delete_By_Search(TrackList):
  if TrackList.isEmpty():
    print("트랙 리스트가 비어 있습니다.")
    return
  index = Search(TrackList)
  result = TrackList.dequeue(index)
  print(f"{result[0]} {result[1]} 가 삭제되었습니다.")

In [ ]:
def user_main():
  TrackList = Queue()
  while True:
    print("*"*30)
    print(" c: 추가 \n p: 재생 \n s: 검색 \n d: 제거 \n q: 종료")
    print("*"*30)
    function = input("무엇을 하고자 하십니까? ")
    if function == "q":
      break
    elif function == "c":
      Create(Ranking, TrackList)
    elif function == "p":
      Play(TrackList)
    elif function == "s":
      Search(TrackList)
    elif function == "d":
      Delete_By_Search(TrackList)
    else:
      print("잘못된 입력입니다.")

In [ ]:
user_main()